In [376]:
# Отключение всех предупреждений.
import warnings
warnings.filterwarnings("ignore")

# Импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt import Trials
import optuna

In [377]:
# Импортируем датасет.
data = pd.read_csv('data/_train_sem09 (1).csv')
data

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


In [378]:
# Разделяем данные на целевую метрику и факторы.
X = data.drop(['Activity'], axis=1)
y = data['Activity']

# Разбиваем данные на обучающую и тестовую выборки.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

In [379]:
# Для начала обучим линейную модель с параметрами по-умолчанию для сравнения с последующими результатами.
# Создаем объект класса логистическая регрессия.
log_reg = linear_model.LogisticRegression(max_iter = 50)

# Обучение модели логистической регрессии.
log_reg.fit(X_train, y_train)

# Делаем предсказание модели на тестовой выборке и выводим метрику f1_score.
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.79


In [380]:
# Поиск оптимальных параметров логистической регрессии с помощью поиска по сетке GridSearchCV.

# Задаем сетку параметров поиска для GridSearchCV.
param_grid = {'penalty': ['l1', 'l2', 'none'], # тип регурялизации
              'solver': ['lbfgs', 'liblinear', 'sag'], # алгоритм оптимизации
              'C': list(np.linspace(0.01, 1, 10, dtype=float)), # уровень силы регурялизации
              }

# Производим поиск оптимальных параметров с использованием поиска по сетке GridSearchCV.
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, # генератор случайных чисел
        max_iter=10 # количество итераций на сходимость
    ),
    param_grid=param_grid, 
    cv=5, # количество фолдов кросс-валидации.
    n_jobs = -1
)
# Обучение модели логистической регрессии.
grid_search.fit(X_train, y_train)

# Делаем предсказание модели, выводим метрику f1_score и оптимальные параметры.
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.23, 'penalty': 'l1', 'solver': 'liblinear'}


In [381]:
# Поиск оптимальных параметров логистической регрессии при помощи рандомизированного поиска RandomizedSearchCV.

# Задаем словарь параметров для рандомизированного поиска RandomizedSearchCV.
param_distributions = {'penalty': ['l1', 'l2', 'none'], # тип регурялизации
              'solver': ['lbfgs', 'liblinear', 'sag'], # алгоритм оптимизации
              'C': list(np.linspace(0.01, 1, 10, dtype=float)), # уровень силы регурялизации
              }

# Поиск оптимальных параметров при помощи RandomizedSearchCV.
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42,
        max_iter=50),
    param_distributions=param_distributions, 
    cv=5, # количество фолдов кросс-валидации.
    n_iter = 10, # количество итераций.
    n_jobs = -1 # Использование всех доступных ядер процессора.
)

# Обучаем модель на найденных оптимальных параметрах.
random_search.fit(X_train, y_train)

# Делаем предсказание модели, выводим метрику f1_score и оптимальные параметры.
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'liblinear', 'penalty': 'l2', 'C': 0.12}


In [382]:
# Обучим модель случайного леса RandomForestClassifier с параметрами по-умолчанию для сравнения с последующими результатами.

# Создаем объект класса случайного леса.
dt = ensemble.RandomForestClassifier(random_state=42)

# Обучаем модель случайного леса.
dt.fit(X_train, y_train)

# Делаем предсказание модели, выводим метрику f1_score и оптимальные параметры.
y_test_pred = dt.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Test: 0.81


In [383]:
# Поиск оптимальных параметров для RandomForestClassifier при помощи поиска по сетке GridSearchCV.

# Задаем словарь параметров случайного леса для поиска по сетке GridSearchCV.
param_grid = {'n_estimators': list(range(80, 200, 30)), # Количество деревьев решений.
              'min_samples_leaf': [5], # Минимальное количество листьев.
              'max_depth': list(np.linspace(20, 40, 5, dtype=int)) # Максимальная глубина деревьев.
              }
# Поиск оптимальных параметров при помощи GridSearchCV для случайного леса.
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, # Параметры для поиска.
    cv=5, # количество фолдов кросс-валидации.
    n_jobs = -1 # Использование всех доступных ядер процессора.
)
# Обучение модели на найденных оптимальных параметрах.
grid_search_forest.fit(X_train, y_train) 

# Делаем предсказание, выводим метрику f1_score и оптимальные параметры.
y_test_pred = grid_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 140}


In [384]:
# Поиск оптимальных параметров случайного леса RandomForestClassifier при помощи рандомизированного поиска RandomizedSearchCV.

# Задаем словарь параметров случайного леса для поиска при помощи рандомизированного поиска RandomizedSearchCV.
param_grid = {'min_samples_leaf': list(np.linspace(5, 100, 50, dtype=int)), # Минимальное количество листьев.
              'max_depth': list(np.linspace(1, 30, 50, dtype=int)), # Максимальная глубина деревьев.
              'criterion':['entropy','gini'] # # Список коэффициентов алгоритма.
              }
# Поиск оптимальных параметров с помощью рандомизированного поиска RandomizedSearchCV.
random_search_tree = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions = param_grid, # Параметры для поиска.
    cv=5, # количество фолдов кросс-валидации.
    n_iter = 50, # Количество итераций.
    n_jobs = -1 # Использование всех доступных ядер процессора.
)
# Обучение модели на найденных оптимальных параметрах.
random_search_tree.fit(X_train, y_train) 

# Делаем предсказание, выводим метрику f1_score и оптимальные параметры.
y_test_pred = random_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_tree.best_params_))

f1_score на тестовом наборе: 0.81
Наилучшие значения гиперпараметров: {'min_samples_leaf': 8, 'max_depth': 18, 'criterion': 'entropy'}


In [385]:
# Поиск оптимальных параметров для случайного леса RandomForestClassifier при помощи алгоритма Hyperopt.

# Задаем пространство поиска гиперпараметров для случайного леса.
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

# Фиксируем random_state.
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
    # Построение модели.
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)
    # обучаем модель
    model.fit(X, y)
    # Высчитываем метрику F1.
    score = metrics.f1_score(y, model.predict(X))
    # Минимизируем метрику.
    return -score


# Подбор гиперпараметров для RandomForestClassifier.

# Логирование результатов.
trials = Trials()

# Поиск наилучших гиперпараметров.
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print('Наилучшие значения гиперпараметров {}'.format(best))

100%|██████████| 20/20 [00:51<00:00,  2.59s/trial, best loss: -0.9911233547597184]
Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}


In [386]:
# Рассчитаем точность для тестовой выборки случайного леса RandomForestClassifier с параметрами, найденными при помощи алгоритма Hyperopt.
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
# Делаем предсказание и выводим метрику F1.
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [387]:
# Поиск оптимальных параметров для случайного леса RandomForestClassifier при помощи алгоритма Optuna.

# Фиксируем random_state
random_state = 42
def optuna_rf(trial):
    # Задаем пространства поиска гиперпараметров.
    n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
    max_depth = trial.suggest_int('max_depth', 10, 30, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

    # Создаем объект класса случайного леса.
    model = ensemble.RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        min_samples_leaf = min_samples_leaf,
        random_state = random_state
    )
    #  Обучаем модель.
    model.fit(X_train, y_train)
    
    # Вычисляем метрику F1 на обучающей выборке.
    y_train_predict = model.predict(X_train)
    score = metrics.f1_score(y_train, y_train_predict)

    return score

# Создаем объект исследования.
# Указываем, что нам нужно максимизировать метрику direction = 'maximize'.
study = optuna.create_study(study_name = 'RandomForestClassifier', direction = 'maximize')

# Находим лучшую комбинацию гиперпараметров.
study.optimize(optuna_rf, n_trials = 20)

# Выводим результаты на обучающей выборке.
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print('f1_score на обучающем наборе: {:.2f}'.format(study.best_value))

[I 2023-04-29 22:05:40,798] A new study created in memory with name: RandomForestClassifier
[I 2023-04-29 22:05:43,561] Trial 0 finished with value: 0.8904571954587297 and parameters: {'n_estimators': 199, 'max_depth': 12, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.8904571954587297.
[I 2023-04-29 22:05:45,682] Trial 1 finished with value: 0.8803183348637893 and parameters: {'n_estimators': 159, 'max_depth': 10, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8904571954587297.
[I 2023-04-29 22:05:47,321] Trial 2 finished with value: 0.9165902841429879 and parameters: {'n_estimators': 104, 'max_depth': 13, 'min_samples_leaf': 7}. Best is trial 2 with value: 0.9165902841429879.
[I 2023-04-29 22:05:49,871] Trial 3 finished with value: 0.9590464547677262 and parameters: {'n_estimators': 137, 'max_depth': 30, 'min_samples_leaf': 4}. Best is trial 3 with value: 0.9590464547677262.
[I 2023-04-29 22:05:52,498] Trial 4 finished with value: 0.9914163090128756 and parameters: {'n

Наилучшие значения гиперпараметров {'n_estimators': 131, 'max_depth': 23, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.99


In [388]:
# Создаем объект случайного леса с наилучшими найденными параметрами при помощи алгоритма Optuna. 
model = ensemble.RandomForestClassifier(**study.best_params, random_state = random_state)

# Обучаем модель на оптимальных параметрах.
model.fit(X_train, y_train)

# Вычисляем метрику F1 на тестовой выборке.
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.81
